In [6]:
# 2025/6/25
# zhangzhong
# structured output

In [7]:
from utils import load_env 
load_env()

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

# class Classification(BaseModel):
#     sentiment: str = Field(description="The sentiment of the text")
#     aggressiveness: int = Field(
#         description="How aggressive the text is on a scale from 1 to 10"
#     )
#     language: str = Field(description="The language the text is written in")

# https://smith.langchain.com/o/259611c1-6f1c-5608-9fbd-0e6ae9d5e348/projects/p/1b7323ef-7d13-4a10-915f-2c43189ddf97?timeModel=%7B%22duration%22%3A%227d%22%7D&peek=f4a7adf8-dbd5-4510-991a-eaf77072eec8&peeked_trace=f4a7adf8-dbd5-4510-991a-eaf77072eec8&runtab=2
# 从langsmith来看，这个classification会转换成function call, 具体内部是怎么实现的呢？
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )
# 哦，我懂了，结构化输出其实就是利用了function call可以提取函数参数的能力
# 我们想要的结构化的输出其实就是函数的参数！
# 所以想要训练出支持functioncall的模型，实际上就是给定一个函数定义，然后标注他的结果就行了！

In [9]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

structured_llm = model.with_structured_output(Classification)

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='happy', aggressiveness=1, language='spanish')